In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
# checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
# transforming images
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),   #change pixel range from 0-255 to 0-1 and also changes the datatype from numpy to tensor
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5])  
])

In [ ]:
# loading data

train_path='/content/gdrive/MyDrive/Seminar_AppleDetection'
test_path='/content/gdrive/MyDrive/Seminar_AppleDetection'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=256,shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=256,shuffle=True
)

In [ ]:
# categories
root = pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['fresh_apple', 'rotten_apple']


In [ ]:
#CNN Network
class ConvNet(nn.Module):
  def __init__(self,num_classes=2):
    super(ConvNet,self).__init__()
    

    #input shape=(256,3,150,150)
    self.conv1=nn.Conv2d(3,12,3,stride=1,padding=1)
    # new shape = (256,12,150,150)
    self.bn1=nn.BatchNorm2d(num_features=12)
    self.relu1=nn.ReLU()

    self.pool=nn.MaxPool2d(2)
    #reduce the image size be factor 2
    #shape=(256,12,75,75)

    self.conv2=nn.Conv2d(12,20,3,stride=1,padding=1)
    #shape=(256,20,75,75)
    self.relu2=nn.ReLU()

    self.conv3=nn.Conv2d(20,32,3,stride=1,padding=1)
    # new shape = (256,32,75,75)
    self.bn3=nn.BatchNorm2d(num_features=32)
    self.relu3=nn.ReLU()

    self.fc=nn.Linear(in_features=32*75*75,out_features=num_classes)
    
    # Feed Forward Function

  def forward(self,input):
    output=self.conv1(input)
    output=self.bn1(output)
    output=self.relu1(output)

    output=self.pool(output)

    output=self.conv2(output)
    output=self.relu2(output)

    output=self.conv3(output)
    output=self.bn3(output)
    output=self.relu3(output)

      #above output will be in matrix form,with shape(256,32,75,75)

    output=output.view(-1,32*75*75)
    output=self.fc(output)

    return output


In [ ]:
model = ConvNet(2).to(device)

In [ ]:
# optimizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

In [ ]:
#calculating the size of trainiing and testing images
train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))

In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(9.1685) Train Accuracy: 0.7078961899503037 Test Accuracy: 0.8663721700717836
Epoch: 1 Train Loss: tensor(5.0275) Train Accuracy: 0.8343456653782441 Test Accuracy: 0.7261181667586969
Epoch: 2 Train Loss: tensor(3.2779) Train Accuracy: 0.8735505245720596 Test Accuracy: 0.8652678078409718
Epoch: 3 Train Loss: tensor(1.6997) Train Accuracy: 0.913859745996687 Test Accuracy: 0.9122032026504694
Epoch: 4 Train Loss: tensor(1.0376) Train Accuracy: 0.9177250138045279 Test Accuracy: 0.8310325786858089
Epoch: 5 Train Loss: tensor(0.9572) Train Accuracy: 0.9105466593042518 Test Accuracy: 0.9458862506902264
Epoch: 6 Train Loss: tensor(0.7171) Train Accuracy: 0.9226946438431806 Test Accuracy: 0.9442297073440088
Epoch: 7 Train Loss: tensor(0.6894) Train Accuracy: 0.9254555494202098 Test Accuracy: 0.9552733296521259
Epoch: 8 Train Loss: tensor(0.3268) Train Accuracy: 0.9580342352291552 Test Accuracy: 0.9530646051905025
Epoch: 9 Train Loss: tensor(0.2924) Train Accuracy: 0.95